**CMLS - Homework 1 - Assignment 1: Musical Genre Classification**

Implement a classifier able to predict the musical genre of a music piece.

In [62]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp

from pathlib import Path

**Features Computation**

In [63]:
#We give as input the complete audio signal, without framing it

def compute_mfcc(audio, fs, n_mfcc):
    # First of all we code the first 2 steps: windowing and DFT
    # Compute the spectrogram of the audio signal
    X = np.abs(librosa.stft(
        audio,
        window='hamming',
        n_fft=1024,
        hop_length=512,)
        )
    
    # 3rd step: Find the weights of the mel filters
    mel = librosa.filters.mel(
        # Sampling frequency
        sr=fs,
        # Number used for the FFT
        n_fft=1024,
        # How many filters we want
        n_mels=40,
        # Minimum frequency on Mel Scale
        fmin=133.33,
        # Maximum frequency on Mel Scale
        fmax=6853.8,
        # These are just standard values used in the Mel Scale
    )
    
    # Apply the filters to spectrogram to get Y(m)
    melspectrogram = np.dot(mel, X)
    # Take the logarithm (I sum a small quantity in order to avoid log(0))
    log_melspectrogram = np.log10(melspectrogram + 1e-16)
    
    # Apply the DCT to log melspectrogram to obtain the coefficients
    mfcc = sp.fftpack.dct(log_melspectrogram, axis=0, norm='ortho')[1 : n_mfcc + 1]
    # The last parameter is to choose how many coefficients I want to use
    return mfcc

In [89]:
audiofiles = [str(file) for file in Path().glob('genres/jazz/train/*.wav')]
print(np.shape(audiofiles));

(66,)


**Features Computation**: training

In [86]:
classes = ['classical', 'disco', 'jazz', 'country']
n_mfcc = 16
dict_train_features = {'classical': [], 'disco': [], 'jazz': [], 'country': []}

for c in classes:
    #train_root = 'genres/{}/'.format(c)           <--- split genres into subfolders before doing this
    train_root = 'genres/jazz/train/'.format(c)    #only for jazz genre
    class_train_files = [f for f in os.listdir(train_root) if f.endswith('.wav')]
    n_train_samples = len(class_train_files)
    
    train_features = np.zeros((n_train_samples, n_mfcc))
    
    for index, f in enumerate(class_train_files):
        audio, fs = librosa.load(os.path.join(train_root, f), sr=None)
        mfcc = compute_mfcc(audio, fs, n_mfcc)
        train_features[index, :] = np.mean(mfcc, axis=1)
        
    dict_train_features[c] = train_features
    
    print(np.shape(train_features))

(66, 16)
(66, 16)
(66, 16)
(66, 16)


In [80]:
dict_test_features = {'classical': [], 'disco': [], 'jazz': [], 'country': []}

for c in classes:
    test_root = 'genres/jazz/test/'.format(c)
    class_test_files = [f for f in os.listdir(test_root) if f.endswith('.wav')]
    n_test_samples = len(class_test_files)
    
    test_features = np.zeros((n_test_samples, n_mfcc))
    for index, f in enumerate(class_test_files):
        audio, fs = librosa.load(os.path.join(test_root, f), sr=None)
        mfcc = compute_mfcc(audio, fs, n_mfcc)
        test_features[index, :] = np.mean(mfcc, axis=1)
    dict_test_features[c] = test_features
    
    print(np.shape(test_features));

(34, 16)
(34, 16)
(34, 16)
(34, 16)


In [81]:
#Better understanding of this!!
#Maybe choose another plotting method

for c in classes:
    mfcc = dict_train_features[c].transpose()
    # Visualization
    fig = plt.figure(figsize=(16, 6))
    plt.subplot(1,2,1)
    plt.imshow(mfcc, origin='lower', aspect='auto')
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.title('MFCC (coefficients 0 to 13) for class {}'.format(c))
    plt.colorbar()
    plt.tight_layout()

    mfcc_upper = mfcc[7:]
    plt.subplot(1,2,2)
    plt.imshow(mfcc_upper, origin='lower', aspect='auto')
    plt.title('MFCC (coefficients 7 to 13) for class {}'.format(c))
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.colorbar()
    plt.tight_layout()

In [69]:
#SVM

class_0 = 'classical'
class_1 = 'disco'
class_2 = 'jazz'
class_3 = 'country'

x_train_0 = dict_train_features[class_0]
x_train_1 = dict_train_features[class_1]
x_train_2 = dict_train_features[class_2]
x_train_3 = dict_train_features[class_3]

y_train_0 = np.zeros((x_train_0.shape[0],))
y_train_1 = np.ones((x_train_1.shape[0],))
y_train_2 = np.ones((x_train_2.shape[0],))*2
y_train_3 = np.ones((x_train_3.shape[0],))*3

y_train = np.concatenate((y_train_0, y_train_1, y_train_1, y_train_3), axis=0)

In [70]:
x_test_0 = dict_test_features[class_0]
x_test_1 = dict_test_features[class_1]
x_test_2 = dict_test_features[class_2]
x_test_3 = dict_test_features[class_3]

y_test_0 = np.zeros((x_test_0.shape[0],))
y_test_1 = np.ones((x_test_1.shape[0],))
y_test_2 = np.ones((x_test_2.shape[0],))*2
y_test_3 = np.ones((x_test_3.shape[0],))*3

y_test = np.concatenate((y_test_0, y_test_1, y_test_2, y_test_3), axis=0)

In [71]:
#Normalize features

feat_max = np.max(np.concatenate((x_train_0, x_train_1, x_train_2, x_train_3), axis=0), axis=0)
feat_min = np.min(np.concatenate((x_train_0, x_train_1, x_train_2, x_train_3), axis=0), axis=0)

x_train_0_normalized = (x_train_0 - feat_min) / (feat_max - feat_min)
x_train_1_normalized = (x_train_1 - feat_min) / (feat_max - feat_min)
x_train_2_normalized = (x_train_2 - feat_min) / (feat_max - feat_min)
x_train_3_normalized = (x_train_3 - feat_min) / (feat_max - feat_min)

x_test_0_normalized = (x_test_0 - feat_min) / (feat_max - feat_min)
x_test_1_normalized = (x_test_1 - feat_min) / (feat_max - feat_min)
x_test_2_normalized = (x_test_2 - feat_min) / (feat_max - feat_min)
x_test_3_normalized = (x_test_3 - feat_min) / (feat_max - feat_min)

x_test_normalized = np.concatenate((x_test_0_normalized, x_test_1_normalized, x_test_2_normalized, x_test_3_normalized), axis=0)

In [72]:
#Define and train a model for each couple of classes

SVM_parameters={
    'C': 1,
    'kernel': 'rbf',
}

clf_01 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_02 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_03 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_12 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_13 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_23 = sklearn.svm.SVC(**SVM_parameters, probability=True)

In [73]:
clf_01.fit(np.concatenate((x_train_0_normalized, x_train_1_normalized), axis=0), 
           np.concatenate((y_train_0, y_train_1), axis=0))
           
clf_02.fit(np.concatenate((x_train_0_normalized, x_train_2_normalized), axis=0), 
           np.concatenate((y_train_0, y_train_2), axis=0))

clf_03.fit(np.concatenate((x_train_0_normalized, x_train_3_normalized), axis=0), 
           np.concatenate((y_train_0, y_train_3), axis=0))

clf_12.fit(np.concatenate((x_train_1_normalized, x_train_2_normalized), axis=0), 
           np.concatenate((y_train_1, y_train_2), axis=0))

clf_13.fit(np.concatenate((x_train_1_normalized, x_train_3_normalized), axis=0), 
           np.concatenate((y_train_1, y_train_3), axis=0))

clf_23.fit(np.concatenate((x_train_2_normalized, x_train_3_normalized), axis=0), 
           np.concatenate((y_train_2, y_train_3), axis=0))

SVC(C=1, probability=True)

In [74]:
y_test_predicted_01 = clf_01.predict(x_test_normalized).reshape(-1, 1)
y_test_predicted_02 = clf_02.predict(x_test_normalized).reshape(-1, 1)
y_test_predicted_03 = clf_03.predict(x_test_normalized).reshape(-1, 1)
y_test_predicted_12 = clf_12.predict(x_test_normalized).reshape(-1, 1)
y_test_predicted_13 = clf_13.predict(x_test_normalized).reshape(-1, 1)
y_test_predicted_23 = clf_23.predict(x_test_normalized).reshape(-1, 1)

In [75]:
#Majority voting

y_test_predicted = np.concatenate((y_test_predicted_01, y_test_predicted_02, y_test_predicted_03, y_test_predicted_12, y_test_predicted_13, y_test_predicted_23), axis=1)
y_test_predicted = np.array(y_test_predicted, dtype=np.int)

y_test_predicted_mv = np.zeros((y_test_predicted.shape[0],))
for i, e in enumerate(y_test_predicted):
    y_test_predicted_mv[i] = np.bincount(e).argmax() 
    
    

**Confusion Matrix computation**

In [76]:
def compute_multiclass(gt, predicted):
    classes = np.unique(gt)
    
    CM = np.zeros((len(classes), len(classes)))
    
    for i in np.arange(len(classes)):
        pred_class = predicted[gt==i]
        
        for j in np.arange(len(pred_class)):
            CM[i, int(pred_class[j])] = CM[i, int(pred_class[j])] + 1
            
    print(CM)

In [77]:
#Evaluation metrics

def compute_metrics(gt_labels, predicted_labels):
    TP = np.sum(np.logical_and(predicted_labels == 1, gt_labels == 1))
    FP = np.sum(np.logical_and(predicted_labels == 1, gt_labels == 0))
    TN = np.sum(np.logical_and(predicted_labels == 0, gt_labels == 0))
    FN = np.sum(np.logical_and(predicted_labels == 0, gt_labels == 1))
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1_score = 2 * precision * recall / (precision + recall)
    print("Results : \n accuracy = {} \n precision = {} \n recall = {} \n F1 score = {}".format(
        accuracy, precision, recall, F1_score))

In [78]:
compute_multiclass(y_test, y_test_predicted_mv)

print(np.shape(y_test))
print(y_test)
print(np.shape(y_test_predicted_mv))
print(y_test_predicted_mv)

[[13.  0.  0. 21.]
 [13.  0.  0. 21.]
 [13.  0.  0. 21.]
 [13.  0.  0. 21.]]
(136,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
(136,)
[3. 0. 3. 3. 0. 3. 3. 3. 0. 3. 0. 3. 3. 0. 0. 3. 3. 3. 0. 0. 3. 3. 0. 3.
 0. 3. 3. 0. 0. 3. 3. 3. 0. 3. 3. 0. 3. 3. 0. 3. 3. 3. 0. 3. 0. 3. 3. 0.
 0. 3. 3. 3. 0. 0. 3. 3. 0. 3. 0. 3. 3. 0. 0. 3. 3. 3. 0. 3. 3. 0. 3. 3.
 0. 3. 3. 3. 0. 3. 0. 3. 3. 0. 0. 3. 3. 3. 0. 0. 3. 3. 0. 3. 0. 3. 3. 0.
 0. 3. 3. 3. 0. 3. 3. 0. 3. 3. 0. 3. 3. 3. 0. 3. 0. 3. 3. 0. 0. 3. 3. 3.
 0. 0. 3. 3. 0. 3. 0. 3. 3. 0. 0. 3. 3. 3. 0. 3.]
